# Ciencia de Datos en Python - Proyecto #1

El proyecto consiste en aplicar los conocimientos aprendidos en clase (y apoyandose de referencias adicionales utiles) para crear modelos predictivos de regresi´on lineal uni-variable sencillos de la forma:



$$
y = β0 + β1 ∗ x
$$

Donde:
* y es la variable dependiente
* x es la variable independiente
* β0 es el intercepto de la recta
* β1 es la pendiente de la recta

#### 1. Usando sclicing con NumPy separar los datos en 2 datasets: entrenamiento(80 %) y validacion y pruebas(20 %)

In [ ]:
import numpy as np

# Carga del archivo de entrenamiento
data = np.load('./proyecto_training_data.npy')

# buscamos obtner el 80% de la longitud del conjunto de datos
train_size = int(len(data) * 0.8)

# Utlizando slicing separamos el 80% de entrenamiento y 20% de validacion
train_data = data[:train_size]
validation_data = data[train_size:]

#print(train_size)
#print(validation_data)
#validation_data

print(f"Conjunto de entrenamiento: {train_data.shape}")
print(f"Conjunto de validación/pruebas: {validation_data.shape}")

#### 3. Analisis exploratorio de datos: Para cada variable en el dataset calcular((usando numpy o pandas):
* media
* valor maximo
* valor mınimo
* rango(peak to peak, no el rango del tensor que por ser vector sabemos que es 1)
* desviacion estandar.

In [ ]:
def calculate_statistics(data):
    statistics = []
    for i in range(data.shape[1]):
        column = data[:, i][~np.isnan(data[:, i])]
        mean = np.mean(column)  # Media
        max_value = np.max(column)  # Valor máximo
        min_value = np.min(column)  # Valor mínimo
        range_value = np.ptp(column)  # Rango (peak to peak)
        std_dev = np.std(column)  # Desviación estándar
        
        # Agregar las estadísticas calculadas a la lista
        statistics.append((mean, max_value, min_value, range_value, std_dev))
    return statistics

# Calcular las estadísticas para los conjuntos de entrenamiento y de validación/pruebas
train_statistics = calculate_statistics(train_data) # data entrenamiento 80%
validation_statistics = calculate_statistics(validation_data) # data validacion 20%

# Imprimir las estadísticas para el conjunto de entrenamiento
print("Estadísticas del conjunto de entrenamiento:")
for i, stats in enumerate(train_statistics):
    print(f"Variable {i+1}: Media={stats[0]}, Máximo={stats[1]}, Mínimo={stats[2]}, Rango={stats[3]}, Desviación estándar={stats[4]}")

# Imprimir las estadísticas para el conjunto de validación/pruebas
print("\nEstadísticas del conjunto de validación/pruebas:")
for i, stats in enumerate(validation_statistics):
    print(f"Variable {i+1}: Media={stats[0]}, Máximo={stats[1]}, Mínimo={stats[2]}, Rango={stats[3]}, Desviación estándar={stats[4]}")

#### 4. Para cada variable en el dataset usar seaborn(funcion distplot https://seaborn.pydata. org/generated/seaborn.distplot.html) para graficar un histograma de la variable.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)


# ['SalePrice', 'OverallQual', '1stFlrSF', 'TotRmsAbvGrd', 'YearBuilt', 'LotFrontage']
variable_names = ['SalePrice', 'OverallQual', '1stFlrSF', 'TotRmsAbvGrd', 'YearBuilt', 'LotFrontage']

sns.set(style="whitegrid")


# Crear un histograma para cada variable
for i, var_name in enumerate(variable_names):
    plt.figure(figsize=(10, 6))  # Tamaño de la figura
    # Filtrar NaNs y utilizar histplot para la visualización
    sns.displot(train_data[:, i][~np.isnan(train_data[:, i])], kde=False, color='blue')
    #sns.histplot(train_data[:, i][~np.isnan(train_data[:, i])], bins=30, kde=False, color='blue')
    plt.title(f'Histograma de {var_name}')
    plt.xlabel(var_name)
    plt.ylabel('Frecuencia')
    plt.show()

#### 5. Para cada variable independiente x :
* Calcular el coeficiente de correlacion entre x y y.
* Graficar x vs y(scatterplot) usando matplotlib.
* Colocar el coeficiente de correlaci´on y colocarlo como parte del tıtulo de la grafica.
* Basado en la gr´afica y el coeficiente de correlacion de cada par x,y elegir las 2 variables con m´as potencial predictivo es decir las 2 variables que presentan mayor correlaci´on entre dicha variable y la variable dependiente.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Variables independientes
variable_names = ['OverallQual', '1stFlrSF', 'TotRmsAbvGrd', 'YearBuilt', 'LotFrontage']

# SalePrice es la variable dependiente
y = train_data[:, 0]

for i, var_name in enumerate(variable_names, 1):
    x = train_data[:, i]
    valid_index = ~np.isnan(x)
    correlation = np.corrcoef(y[valid_index], x[valid_index])[0, 1]
    
    plt.figure(figsize=(10, 6))
    plt.scatter(x[valid_index], y[valid_index], color='blue')
    plt.title(f"{var_name} vs SalePrice (Correlación: {correlation:.2f})")
    plt.xlabel(var_name)
    plt.ylabel('SalePrice')
    plt.show()

#### 6. Crear una funcion para entrenar un modelo de regresion lineal de una variable y = β0 +β1 ∗x.
La funcion recibe como argumentos:
* 6.1 Vector con la variable independiente x,
* 6.2 Vector con la variable dependiente y,
* 6.3 un entero epochs que indica por cuantas iteraciones entrenar el modelo.
* 6.4 un entero imprimir error cada , que nos indica cada cuantas iteraciones queremos imprimir a traves de print: el numero de iteracion, el error del modelo en esa iteracion, si imprimir error cada = 10, se despliega en pantalla el error en las iteraciones:
10,20,30,40,50.
* 6.5 escalar α(learning rate): es usado como parte de la expresi´on matem´atica para actualizar en cada iteraci´on los par´ametros del modelo.

In [ ]:
import numpy as np

def entrenar_regresion_lineal(x, y, epochs, imprimir_error_cada, alpha):
    beta_0 = 0  # Inicialización de parámetro
    beta_1 = 0  # Inicialización de parámetro

    for epoch in range(epochs):
        y_pred = beta_0 + beta_1 * x  # Calcula la predicción
        error = y - y_pred  # Calcula el error
        cost = (error**2).mean()  # Calcula el costo

        if epoch % imprimir_error_cada == 0:
            print(f"Iteración {epoch}, Error {cost}")

        # Actualiza los parámetros
        beta_0 -= alpha * (-2 * error.mean())
        beta_1 -= alpha * (-2 * (x * error).mean())

    return beta_0, beta_1